In [0]:
# Libs
from pyspark.sql.functions import col, to_date, year, month, dayofmonth, date_format

# Paths Silver layer
path_silver_profile = "abfss://silver@datalakeetlproject.dfs.core.windows.net/profile"
path_silver_historical = "abfss://silver@datalakeetlproject.dfs.core.windows.net/historical"

# Paths Gold layer
path_gold_company = "abfss://gold@datalakeetlproject.dfs.core.windows.net/dim_company"
path_gold_date = "abfss://gold@datalakeetlproject.dfs.core.windows.net/dim_date"
path_gold_fact = "abfss://gold@datalakeetlproject.dfs.core.windows.net/fact_quote"

# Read data from Silver layer
df_profile = spark.read.format("delta").load(path_silver_profile)
df_historical = spark.read.format("delta").load(path_silver_historical)

In [0]:
#Datalake connection
blobAccessKey = dbutils.secrets.get(scope = "myscope", key = "accesskey")
spark.conf.set("fs.azure.account.key.datalakeetlproject.dfs.core.windows.net", 
               blobAccessKey) 

In [0]:
# Create dim_company in Gold layer
dim_company = df_profile.select(
    col("ticker"),
    col("company_name"),
    col("sector"),
    col("industry"),
    col("country"),
    col("ceo_name"),
    col("exchange"),
    col("market_cap"),
    col("website")
).dropDuplicates(["ticker"])


In [0]:
# Create dim_date in Gold layer
dim_date = df_historical.select(
    to_date("date", "yyyy-MM-dd").alias("date"),
).dropDuplicates() \
 .withColumn("year", year(col("date"))) \
 .withColumn("month", month(col("date"))) \
 .withColumn("day", dayofmonth(col("date"))) \
 .withColumn("week_day", date_format(col("date"), "E"))


In [0]:
# Create fact_quote in Gold layer
fact_quote = df_historical.select(
    col("ticker"),
    to_date("date", "yyyy-MM-dd").alias("date"),
    col("open"),
    col("close"),
    col("high"),
    col("low"),
    col("volume")
)

In [0]:
# Save data in Gold layer
dim_company.write.mode("overwrite").format("delta").save(path_gold_company)
dim_date.write.mode("overwrite").format("delta").save(path_gold_date)
fact_quote.write.mode("overwrite").format("delta").save(path_gold_fact)
